In [203]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [204]:
articles = pd.read_csv('./data/articles.csv')
customers = pd.read_csv('./data/customers.csv')
transactions_train = pd.read_csv('./data/sept_2020.csv')

In [205]:
print(transactions_train.shape)
transactions_train.head()

(798269, 8)


,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id,year,month
0,30990055,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,777148006,0.013542,1,2020,9
1,30990056,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,835801001,0.018627,1,2020,9
2,30990057,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,923134005,0.012695,1,2020,9
3,30990058,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,865929003,0.016932,1,2020,9
4,30990059,2020-09-01,0005ed68483efa39644c45185550a82dd09acb07622acb...,863646004,0.033881,1,2020,9


In [206]:
# rating as the number of purchases for the item
article_trans = pd.DataFrame(transactions_train.groupby(['customer_id', 'article_id']).count()['t_dat']).reset_index().rename(columns={'t_dat': 'article_rating'})
article_trans.head()

,customer_id,article_id,article_rating
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,794321007,1
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,448509014,1
3,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,719530003,1
4,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,685814001,3


In [207]:
# transactions per item
trans_per_item = pd.DataFrame(transactions_train.groupby(['article_id']).count()['customer_id']).reset_index()
trans_per_item.head()

,article_id,customer_id
0,108775044,10
1,110065011,1
2,111565001,95
3,111586001,250
4,111593001,257


In [208]:
filtered_trans_per_item = trans_per_item[trans_per_item.customer_id >= 20]

In [209]:
popular_articles = filtered_trans_per_item.article_id.tolist()

In [210]:
# transactions per user
trans_per_user = pd.DataFrame(transactions_train.groupby(['customer_id']).count()['article_id']).reset_index()
trans_per_user.head()

,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,1
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,2
3,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,3
4,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,10


In [211]:
filtered_trans_per_user = trans_per_user[trans_per_user.article_id >= 10]

In [212]:
busy_users = filtered_trans_per_user.customer_id.tolist()

In [213]:
# filter transactions to only popular articles and busy users
filtered_trans = article_trans[article_trans.article_id.isin(popular_articles)]
filtered_trans = filtered_trans[filtered_trans.customer_id.isin(busy_users)]

In [214]:
print(filtered_trans.shape)
filtered_trans.head()

(175483, 3)


,customer_id,article_id,article_rating
5,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,572998013,1
9,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,865929003,1
11,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,909869004,1
12,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,923134003,1
13,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,923134005,1


In [215]:
trans_matrix = filtered_trans.pivot_table(index='customer_id', columns='article_id', values='article_rating')
# replace NaN values with 0
trans_matrix = trans_matrix.fillna(0)
# display the top few rows
trans_matrix.head()

article_id,111565001,111586001,111593001,111609001,123173001,129085001,130035001,146730001,148033001,153115019,153115020,153115021,153115040,153115043,156224001,156224002,156227002,156231001,156231002,158340001,160442007,160442010,160442043,176754003,179123001,179208001,179950001,179950002,179950017,182909001,186262001,186262006,186266004,189616001,189616006,189616028,189616032,189626001,194037001,200182001,200182002,201219001,201219003,201219016,201219017,212629047,212629048,214844001,214844002,214844003,215589002,224606019,226959007,226959008,228257001,228257002,228257003,228257004,228257008,235596002,237347045,237347059,237347060,237347063,240561001,243937001,243937028,250099001,250099002,253448001,253448002,253448003,265630004,265630006,266875001,266875006,268305001,270375004,278811006,278811011,282832018,293433002,294008005,294008056,297067002,297078001,297078002,297078008,300024058,301227002,301227019,301656026,305931001,307239004,307239005,307239010,309864002,309864012,309864013,309864016,312878010,316085001,316085002,316441001,316441005,319906009,322017003,322017040,324946001,324963051,333323030,333323038,335037005,337991001,337991009,338016009,341129001,341186001,342060001,347200001,349080001,349301001,351484002,351484039,357751001,357751002,362219001,362219002,372860001,372860002,372860024,372860068,372860069,372860072,372860073,373506001,373506004,373506008,373506027,375917006,379963002,386678001,392938001,395127001,395127006,395127011,395127012,399061008,399061026,399061028,399061032,399061033,399087010,399087021,399087047,399087054,399136004,399136009,399136027,399136061,399201042,399223001,399223052,399256001,399256002,399256005,399256023,399256036,399256037,400285006,403448008,407653001,407653002,408571002,408875001,408875021,408875023,408875028,412370001,417951001,417951005,429313008,433414040,433414041,436083001,436083002,436261001,441386001,441386004,448509001,448509014,448509018,448509026,448509028,448509030,448509031,448509035,449570010,449570041,449570056,449570061,449570064,452618001,452818001,452818031,456163060,456163083,456163084,456163085,456163086,456163087,456163090,457466001,457466002,464297007,464297031,464679015,464679016,464927027,469039019,470789001,470789019,470789030,470789031,470985003,473954008,473954013,473954014,473954015,479167002,480093001,484398001,486639003,486639013,492892002,492897001,492897002,493103034,493103045,497640002,499334001,501722004,501914002,505882002,505882006,506098006,506098007,506098016,507909001,507909003,...,928111001,928152001,928152003,928157002,928158002,928158003,928171001,928206001,928210002,928216002,928331001,928331002,928332001,928351001,928446001,928461001,928461002,928461003,928719001,928723001,928802001,928808001,928813001,928820001,928824001,928835001,928839001,928845001,928857001,928858001,928892001,928892002,928898002,928900001,928905001,928907001,928910001,928912001,928915001,928917003,928995001,929001001,929042001,929165001,929165002,929226002,929226003,929275001,929388002,929397001,929508001,929508002,929511001,929591001,929594001,929599001,929603001,929673001,929689001,929695001,929744001,929745001,929872001,929938002,929980001,929980002,929980006,930058001,930058002,930350001,930350002,930380001,930380003,930533001,930533003,930578001,930829001,930866001,931282001,931335001,931419001,931437001,931696001,931720001,931729004,931769001,931769003,931869001,931981001,932107001,932238001,932243001,932243002,932243003,932365008,932383001,932426001,932578001,932642001,932698001,932798001,932798002,933032001,933032002,933032003,933214001,933327001,933373001,933404001,933408001,933408002,933409001,933409002,933476001,933706001,933802002,933882001,933885004,933889001,933891001,933910002,933928002,933932001,933963001,933989001,933989002,933990001,934053001,934054002,934072001,934114001,934128001,934135001,934211004,934212003,934296001,934312001,934312002,934380001,934536001,934727001,934727002,934793001,934835001,934873001,934873002,934981001,935092001,93519

In [216]:
u = list(filtered_trans[filtered_trans.customer_id=='0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94'].article_id)       
n = list(filtered_trans[filtered_trans.customer_id=='000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed6396773839f6bf71a9'].article_id)
len(list(set(n).intersection(u)))

0

In [217]:
def jaccard_index(user, other_users):
    '''
    input: G-graph, K-number of requested suggested links for each node
    output: .csv file consisting of n lines that have K comma separated values on each line indicating the node index of the
    nodes that are suggested new links that are currently not conncted to that node.
    '''
    similarities = []
    user = list(user.index)[0]
    other_users = list(other_users.index)
    
    for other_user in other_users:
        
        # neighbors of node u
        articles_u = list(filtered_trans[filtered_trans.customer_id==user].article_id)       
                
        # neighbors of node n
        articles_n = list(filtered_trans[filtered_trans.customer_id==other_user].article_id)
        
        # keep nodes that appear in both neighbor lists and get length of remaining nodes
        same_nodes = len(list(set(articles_n).intersection(articles_u)))

        # we divide the above number with the total number of neighbors
        total_neighbors = len(articles_u)

        # append this to the list (common neighbors, node j)
        similarities.append(same_nodes/total_neighbors)
    
    return similarities

In [218]:
from sklearn.metrics.pairwise import cosine_similarity
import operator

# function to find most similar customers (k most similar)
def similar_customers(customer_id, matrix, k=3, sim='cosine'):
    # create a df of just the current user
    user = matrix[matrix.index == customer_id]
    
    # and a df of all other users
    other_users = matrix[matrix.index != customer_id]
    
    if sim == 'cosine':
        # calc cosine similarity between user and each other user
        similarities = cosine_similarity(user,other_users)[0].tolist()
    
    elif sim == 'jaccard':
        similarities = jaccard_index(user, other_users)
    
    # create list of indices of these users
    indices = other_users.index.tolist()
    
    # create key/values pairs of user index and their similarity
    index_similarity = dict(zip(indices, similarities))
    
    # sort by similarity
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    # grab k users off the top
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    scores = [u[1] for u in top_users_similarities]
    
    return users, scores
    
current_user = '0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94'

# try it out
similar_customer_indices, scores = similar_customers(current_user, trans_matrix)
print(similar_customer_indices)
print(scores)

['2a7da7083a8f2c313fe1832034823ab67873d29a26e494c8b5aad53db79eb66c', '02d3b2db21e8b096b453c5c8baa840adace232f7b72feabb1f0db460fd1b415a', 'ccf8134ab4b4fe45d4f58d9262e36f11170ac7f4c8c286db562c0b415f67bb7a']
[0.34403123102809335, 0.33968311024337877, 0.28097574347450816]


In [182]:
# load vectors for similar users
similar_customers = trans_matrix[trans_matrix.index.isin(similar_customer_indices)]
# calc avg ratings across the 3 similar users
similar_customers = similar_customers.mean(axis=0)
# convert to dataframe so its easy to sort and filter
similar_users_df = pd.DataFrame(similar_customers, columns=['mean'])
similar_users_df

# load vector for the current user
user_df = trans_matrix[trans_matrix.index == '0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94']
# transpose it so its easier to filter
user_df_transposed = user_df.transpose()
# rename the column as 'rating'
user_df_transposed.columns = ['rating']
# remove any rows without a 0 value. Article not purchased yet
#user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]
# generate a list of articles the user has not purchased
articles_unpurchased = user_df_transposed.index.tolist()
articles_unpurchased

# filter avg ratings of similar users
similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(articles_unpurchased)]
# order the dataframe
similar_users_df_ordered = similar_users_df_filtered.sort_values(by=['mean'], ascending=False)
similar_users_df_ordered

df = similar_users_df_ordered.merge(user_df_transposed, on='article_id', how='left')
df = df.reset_index()
df

item_rating = []
for item_i in range(len(df.article_id)):
    sum_neighbors_numerator = []
    sum_neighbors_denominator = []
    avg_rating_u = user_df_transposed.rating.mean()
    prediction = 0
    for n in range(len(similar_customer_indices)):
        test = trans_matrix[trans_matrix.index.isin([similar_customer_indices[n]])].T
        test.columns=['rating']
        test = test.reset_index()

        sum_neighbors_numerator.append(scores[n] * (test[test.article_id==test.article_id[item_i]].rating[item_i] - np.mean(test.rating)))
        sum_neighbors_denominator.append(scores[n])

    prediction += avg_rating_u + (np.sum(sum_neighbors_numerator)/np.sum(sum_neighbors_denominator))
    item_rating.append(prediction)

df['item_rating'] = item_rating
similar_users_df_ordered = df.sort_values(by='item_rating', ascending=False)

similar_users_df_ordered

# grab the top n articles   
top_n_articles = similar_users_df_ordered.head(items)
top_n_articles_indices = top_n_articles.article_id.tolist()

,article_id,mean,rating,item_rating
488,891322004,0.0,0.0,2.494987
3755,752689001,0.0,0.0,0.872403
2489,927530006,0.0,0.0,0.711869
4902,599580014,0.0,0.0,0.711869
6237,809411004,0.0,0.0,0.702855
...,...,...,...,...
2218,942955004,0.0,0.0,-0.001378
2217,942955002,0.0,0.0,-0.001378
2216,941005001,0.0,0.0,-0.001378
2215,941005002,0.0,0.0,-0.001378


In [219]:
# make article recommendations for that customer
def recommend_item(user_index, similar_customer_indices, scores, matrix, items=5):
    
    # load vectors for similar users
    similar_customers = matrix[matrix.index.isin(similar_customer_indices)]
    # calc avg ratings across the 3 similar users
    similar_customers = similar_customers.mean(axis=0)
    # convert to dataframe so its easy to sort and filter
    similar_users_df = pd.DataFrame(similar_customers, columns=['mean'])
    
    # load vector for the current user
    user_df = matrix[matrix.index == user_index]
    # transpose it so its easier to filter
    user_df_transposed = user_df.transpose()
    # rename the column as 'rating'
    user_df_transposed.columns = ['rating']
    # remove any rows without a 0 value. Article not purchased yet
    #user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]
    # generate a list of articles the user has not purchased
    articles_unpurchased = user_df_transposed.index.tolist()
    
    # filter avg ratings of similar users
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(articles_unpurchased)]
    # order the dataframe
    similar_users_df_ordered = similar_users_df_filtered.sort_values(by=['mean'], ascending=False)
    
    df = similar_users_df_ordered.merge(user_df_transposed, on='article_id', how='left')
    df = df.reset_index()
    
    item_rating = []
    for item_i in range(len(df.article_id)):
        sum_neighbors_numerator = []
        sum_neighbors_denominator = []
        avg_rating_u = user_df_transposed.rating.mean()
        prediction = 0
        for n in range(len(similar_customer_indices)):
            test = trans_matrix[trans_matrix.index.isin([similar_customer_indices[n]])].T
            test.columns=['rating']
            test = test.reset_index()

            sum_neighbors_numerator.append(scores[n] * (test[test.article_id==test.article_id[item_i]].rating[item_i] - np.mean(test.rating)))
            sum_neighbors_denominator.append(scores[n])
        
        prediction += avg_rating_u + (np.sum(sum_neighbors_numerator)/np.sum(sum_neighbors_denominator))
        item_rating.append(prediction)
    
    df['item_rating'] = item_rating
    similar_users_df_ordered = df.sort_values(by='item_rating', ascending=False)
    
    # grab the top n articles   
    top_n_articles = similar_users_df_ordered.head(items)
    top_n_articles_indices = top_n_articles.article_id.tolist()
    
    # lookup these articles in the other dataframe to find names
    #article_information = articles[articles['article_id'].isin(top_n_articles_indices)]
    
    df = df.dropna(axis=0)
    rmse = np.sqrt((1/df.shape[0]) * np.sum((df.item_rating-df.rating)**2))
    
    # maybe change this to return the list for each customer
    #return article_information #items
    return top_n_articles_indices, rmse

# try it out
customerx_recommend, customerx_rmse = recommend_item('0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94', similar_customer_indices, trans_matrix)
print(customerx_recommend)
print(customerx_rmse)

TypeError: recommend_item() missing 1 required positional argument: 'matrix'

In [193]:
customerx_purchases = filtered_trans[filtered_trans.customer_id=='0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94'].article_id.tolist()
customerx_purchases

[572998013, 865929003, 909869004, 923134003, 923134005, 935858001]

In [194]:
# MAP@5
len(set(customerx_purchases).intersection(set(customerx_recommend)))

0

number of recommendations that are relevant for this customer = 3

number of items we recommended = 5

MAP@5 = 3/5

# Calculate MAP@5

## Using Cosine

Having found that we can successfully recommend 5 items for an individual customer, let's loop through our customers list and find the precision for each customer. This will allow us to find the average of those precisions and see how well the system performs. We can then adjust our similarity scores to see if there is a similarity score that performs better.

In [195]:
list(trans_matrix.head().index)

['0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94',
 '000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed6396773839f6bf71a9',
 '00077dbd5c4a4991e092e63893ccf29294a9d5c46e85010e95f2fc10bf9437a4',
 '000fa1b80857fa40bf25990bc1b1b65afc63923a8e4b5762db34e4bb46032d29',
 '000fb6e772c5d0023892065e659963da90b1866035558ec16fca51b0dcfb7e59']

In [224]:
# map@5 for the filtered list of september 2020 customers
map_5 = []
customer_rmse = []
for i in list(trans_matrix.head().index):
    similar_customer_indices, scores = similar_customers(current_user, trans_matrix)
    customerx_recommend, customerx_rmse = recommend_item(i, similar_customer_indices, scores, trans_matrix)
    customerx_purchases = filtered_trans[filtered_trans.customer_id==i].article_id.tolist()
    map_5.append(len(set(customerx_purchases).intersection(set(customerx_recommend)))/5)

In [226]:
np.mean(customerx_rmse)

0.08672670440428748

In [225]:
np.mean(map_5)

0.0

## Using Jaccard

In [96]:
# map@5 for the filtered list of september 2020 customers
map_5 = []
for i in list(trans_matrix.head().index):
    similar_customer_indices = similar_customers(i, trans_matrix, sim = 'jaccard')
    customerx_recommend = recommend_item(i, similar_customer_indices, trans_matrix)
    customerx_purchases = filtered_trans[filtered_trans.customer_id==i].article_id.tolist()
    map_5.append(len(set(customerx_purchases).intersection(set(customerx_recommend)))/5)

In [97]:
np.mean(map_5)

0.4800000000000001